In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU
# Load the datasets
stock_data_path = '/content/drive/MyDrive/cs5284pro/dow_1day_price.csv'
# etf_data_path = '/content/drivedata/etf_1day_price.csv'

# Read the data
time_series_data = pd.read_csv(stock_data_path)
# etf_data = pd.read_csv(etf_data_path)

# Select the first column for modeling
target_column = time_series_data.columns[0]
target_series = time_series_data[target_column]

# Split data into training and testing sets (e.g., 80% train, 20% test)
train_size = int(len(target_series) * 0.8)
train_data = target_series[:train_size]
test_data = target_series[train_size:]

# Normalize the data
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data.values.reshape(-1, 1))
test_scaled = scaler.transform(test_data.values.reshape(-1, 1))

# Create sequences for time series modeling
def create_sequences(data, time_steps=10):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

time_steps = 10
X_train, y_train = create_sequences(train_scaled, time_steps)
X_test, y_test = create_sequences(test_scaled, time_steps)

# Reshape data for LSTM/GRU
X_train_lstm = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_lstm = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Output the prepared data shapes for verification
X_train.shape, y_train.shape, X_test.shape, y_test.shape



((1980, 10, 1), (1980, 1), (488, 10, 1), (488, 1))

In [7]:
# Train an SVR model
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svr_model.fit(X_train.reshape(X_train.shape[0], -1), y_train.ravel())

# Make predictions using the SVR model
y_pred_svr = svr_model.predict(X_test.reshape(X_test.shape[0], -1))

# Rescale predictions and true values back to the original scale
y_test_rescaled = scaler.inverse_transform(y_test)
y_pred_svr_rescaled = scaler.inverse_transform(y_pred_svr.reshape(-1, 1))

# Evaluate SVR model
svr_mse = mean_squared_error(y_test_rescaled, y_pred_svr_rescaled)
svr_mae = mean_absolute_error(y_test_rescaled, y_pred_svr_rescaled)

svr_mse, svr_mae


(79.58115785930434, 4.38344320784003)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Build the LSTM model
lstm_model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dense(1)  # Output layer
])

# Compile the model
lstm_model.compile(optimizer='adam', loss='mse')

# Train the model
lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32, verbose=0)

# Make predictions using the LSTM model
y_pred_lstm = lstm_model.predict(X_test_lstm)

# Rescale predictions back to the original scale
y_pred_lstm_rescaled = scaler.inverse_transform(y_pred_lstm)
y_test_rescaled = scaler.inverse_transform(y_test)

# Evaluate LSTM model
lstm_mse = mean_squared_error(y_test_rescaled, y_pred_lstm_rescaled)
lstm_mae = mean_absolute_error(y_test_rescaled, y_pred_lstm_rescaled)

lstm_mse, lstm_mae


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step


(2.658832232212019, 1.171209470207965)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU

# Build the GRU model
gru_model = Sequential([
    GRU(50, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dense(1)  # Output layer
])

# Compile the model
gru_model.compile(optimizer='adam', loss='mse')

# Train the model
gru_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32, verbose=0)

# Make predictions using the GRU model
y_pred_gru = gru_model.predict(X_test_lstm)

# Rescale predictions back to the original scale
y_pred_gru_rescaled = scaler.inverse_transform(y_pred_gru)
y_test_rescaled = scaler.inverse_transform(y_test)

# Evaluate GRU model
gru_mse = mean_squared_error(y_test_rescaled, y_pred_gru_rescaled)
gru_mae = mean_absolute_error(y_test_rescaled, y_pred_gru_rescaled)

gru_mse, gru_mae


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step


(3.25513806914114, 1.1307827093343266)